In [1]:
import pandas as pd

df = pd.read_csv('AUCs.txt', sep='\t')
df.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [4]:
from scipy import stats
columns = ['C4.5', 'C4.5+m', 'C4.5+cf', 'C4.5+m+cf']
corr = []

for i, lhs_column in enumerate(columns):
    for j, rhs_column in enumerate(columns):
        if i >= j:
            continue
        
        cor, p = stats.wilcoxon(df[lhs_column], df[rhs_column])
        corr.append([lhs_column, rhs_column, cor, p])
corr

[['C4.5', 'C4.5+m', 6.5, 0.01075713311978963],
 ['C4.5', 'C4.5+cf', 43.0, 0.861262330095348],
 ['C4.5', 'C4.5+m+cf', 11.0, 0.015906444101703374],
 ['C4.5+m', 'C4.5+cf', 17.0, 0.046332729793395394],
 ['C4.5+m', 'C4.5+m+cf', 22.0, 0.3278256758446406],
 ['C4.5+cf', 'C4.5+m+cf', 10.0, 0.022909099354356588]]

In [6]:
for i, item in enumerate(corr):
    if item[3] < 0.05:
        print(item)

['C4.5', 'C4.5+m', 6.5, 0.01075713311978963]
['C4.5', 'C4.5+m+cf', 11.0, 0.015906444101703374]
['C4.5+m', 'C4.5+cf', 17.0, 0.046332729793395394]
['C4.5+cf', 'C4.5+m+cf', 10.0, 0.022909099354356588]


In [10]:
from statsmodels.sandbox.stats.multicomp import multipletests 

correlation = pd.DataFrame.from_records(corr)
correlation.columns = ['clf_A', 'clf_B', 'corr', 'p']
reject, p_corrected, a1, a2 = multipletests(correlation.p, 
                                            alpha = 0.05, 
                                            method = 'holm')

In [11]:
correlation['p_corrected'] = p_corrected
correlation['reject'] = reject
correlation.reject.value_counts()

False    6
Name: reject, dtype: int64

In [12]:
reject, p_corrected, a1, a2 = multipletests(correlation.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh')
correlation['p_corrected'] = p_corrected
correlation['reject'] = reject
correlation.reject.value_counts()

True     3
False    3
Name: reject, dtype: int64